
# Advanced Biostatistical Techniques: PCA, Feature Selection, and SEM

This notebook explores advanced biostatistical techniques, including Principal Component Analysis (PCA), random forest-based feature selection, and Structural Equation Modeling (SEM).

## Dataset Information
The Mice Protein Expression dataset is used, which can be accessed from:
- [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/342/mice+protein+expression)
- [Kaggle](https://www.kaggle.com/datasets/ruslankl/mice-protein-expression)

### Prerequisites
Install the following packages before proceeding:
```bash
pip install pandas numpy scikit-learn matplotlib seaborn semopy graphviz
```
        


## Data Loading and Preprocessing

The dataset contains protein expression levels and associated biological class labels. We'll preprocess the data by imputing missing values and encoding categorical variables.
        

In [ ]:

import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Load dataset (replace with dataset file path)
from ucimlrepo import fetch_ucirepo
mice_protein_expression = fetch_ucirepo(id=342)

# Extract features and targets
X = mice_protein_expression.data.features.iloc[:, :-3]
y = mice_protein_expression.data.targets

# Impute missing values
imputer = SimpleImputer(strategy="mean")
X_imputed = imputer.fit_transform(X)

# Encode target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print(f"Dataset Shape: {X_imputed.shape}")
print(f"Classes: {label_encoder.classes_}")
        


## Principal Component Analysis (PCA)

PCA reduces dimensionality by transforming data into principal components that explain the most variance.
        

In [ ]:

# Perform PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_imputed)

# Create a DataFrame for PCA results
pca_df = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
pca_df["Target"] = y_encoded

# Plot PCA results
plt.figure(figsize=(10, 8))
categories = label_encoder.classes_
colors = plt.cm.viridis(range(len(categories)))

for i, category in enumerate(categories):
    subset = pca_df[pca_df["Target"] == i]
    plt.scatter(subset["PC1"], subset["PC2"], label=category, color=colors[i])

plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.title("PCA of Mice Protein Expression Dataset")
plt.legend(title="Category")
plt.show()
        


## Random Forest for Feature Selection

Random forest identifies the most important features contributing to classification, which helps simplify complex datasets.
        

In [ ]:

from sklearn.ensemble import RandomForestClassifier

# Train random forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_imputed, y_encoded)

# Get top 10 features
feature_importances = pd.Series(rf.feature_importances_, index=X.columns)
top_10_features = feature_importances.nlargest(10).index

# Filter data to include top 10 features
X_top_10 = pd.DataFrame(X_imputed, columns=X.columns)[top_10_features]
print("Top 10 Features:")
print(top_10_features)
        


## Structural Equation Modeling (SEM)

SEM models relationships among latent variables. Here, we analyze latent variables related to cell division and apoptosis signaling pathways.
        

In [ ]:

from semopy import Model, calc_stats, semplot

# Define SEM model
model_desc = '''
Cell_division_signaling =~ pAKT_N + pBRAF_N + pCREB_N
Apoptosis_signaling =~ MTOR_N + P38_N
Cell_division_signaling ~ Apoptosis_signaling
'''

# Fit SEM model
model = Model(model_desc)
model.fit(pd.DataFrame(X_top_10, columns=top_10_features))

# Calculate fit statistics
fit_stats = calc_stats(model)
print(f"CFI: {fit_stats['CFI']}")
        


## Visualizing SEM

The SEM plot shows relationships between latent variables and observed features.
        

In [ ]:

semplot(model, "sem_diagram.png", show="estimates")
plt.imshow(plt.imread("sem_diagram.png"))
plt.axis("off")
plt.title("SEM Path Diagram")
plt.show()
        